# Analyse Data - Fitness Member

### I. Import Required Package and Read Data

In [1]:
import pandas as pd
from dateutil.relativedelta import relativedelta

# Set path for data folder
DATA_PATH: str = "data/3ch/"

# read csv files
customer_master = pd.read_csv(DATA_PATH + "customer_master.csv")
campaign_master = pd.read_csv(DATA_PATH + "campaign_master.csv")
class_master = pd.read_csv(DATA_PATH + "class_master.csv")
use_log = pd.read_csv(DATA_PATH + "use_log.csv")


### II. Merging Data

In [2]:
print(customer_master.columns)
print(campaign_master.columns)
print(class_master.columns)

Index(['customer_id', 'name', 'class', 'gender', 'start_date', 'end_date',
       'campaign_id', 'is_deleted'],
      dtype='object')
Index(['campaign_id', 'campaign_name'], dtype='object')
Index(['class', 'class_name', 'price'], dtype='object')


In [3]:
customer = pd.merge(pd.merge(customer_master, class_master, on="class", how="left"), campaign_master, on="campaign_id", how="left")

In [4]:
customer["start_date"] = pd.to_datetime(customer["start_date"])
customer["end_date"] = pd.to_datetime(customer["end_date"])

### III. Get an Information for Staying Members

#### 1. Check the 'end_data'

- Members can request center to delete their info until the end of each month.
- As a member request, end date of each month will be input in 'end_date'
- Assume that today is 2019-03-25

In [5]:
customer["end_date"].unique()

<DatetimeArray>
[                'NaT', '2018-04-30 00:00:00', '2018-05-31 00:00:00',
 '2018-06-30 00:00:00', '2018-07-31 00:00:00', '2018-08-31 00:00:00',
 '2018-09-30 00:00:00', '2018-11-30 00:00:00', '2018-12-31 00:00:00',
 '2019-01-31 00:00:00', '2018-10-31 00:00:00', '2019-02-28 00:00:00',
 '2019-03-31 00:00:00']
Length: 13, dtype: datetime64[ns]

#### 2. Create Table Excepting Not Null in 'end_date'

In [6]:
# check the data before creating new table
member_stay = customer[customer["end_date"].isnull()]    # True if the member is being stayed.
quit_this_month = customer[customer["end_date"] >= pd.to_datetime("20190331")]

# creating new table
customer_stay = customer[(customer["end_date"].isnull()) | (customer["end_date"] >= pd.to_datetime("20190331"))]
customer_stay

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
0,OA832399,XXXX,C01,F,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반
1,PL270116,XXXXX,C01,M,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반
2,OA974876,XXXXX,C01,M,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반
3,HD024127,XXXXX,C01,F,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반
4,HD661448,XXXXX,C03,F,2015-05-01,NaT,CA1,0,2_야간,6000,2_일반
...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,M,2019-03-14,NaT,CA1,0,0_종일,10500,2_일반
4188,HD246549,XXXXX,C01,F,2019-03-14,NaT,CA1,0,0_종일,10500,2_일반
4189,GD037007,XXXXX,C03,M,2019-03-14,NaT,CA1,0,2_야간,6000,2_일반
4190,OA953150,XXXXX,C01,M,2019-03-14,NaT,CA1,0,0_종일,10500,2_일반


#### 3. Data Analysis by Groupby

In [7]:
tmp = customer_stay[["customer_id", "gender"]].groupby("gender").count()
tmp["ratio"] = tmp["customer_id"] * 100 / len(customer_stay)
tmp

,customer_id,ratio
gender,,
F,1400,47.409414
M,1553,52.590586


In [8]:
tmp = customer_stay[["customer_id", "class_name"]].groupby("class_name").count()
tmp["ratio"] = tmp["customer_id"] * 100 / len(customer_stay)
tmp

,customer_id,ratio
class_name,,
0_종일,1444,48.899424
1_주간,696,23.569252
2_야간,813,27.531324


In [9]:
tmp = customer_stay[["customer_id", "campaign_name"]].groupby("campaign_name").count()
tmp["ratio"] = tmp["customer_id"] * 100 / len(customer_stay)
tmp

,customer_id,ratio
campaign_name,,
0_입회비반액할인,311,10.531663
1_입회비무료,242,8.195056
2_일반,2400,81.273281


#### * Result: The members who were not given any discount tends to stay more longer than the others.
#### Really?

#### 4. Analyse Data for Quitted Members

In [10]:
customer_quit = customer[customer["is_deleted"] == 1]

In [11]:
tmp = customer_quit[["customer_id", "gender"]].groupby("gender").count()
tmp["ratio"] = tmp["customer_id"] * 100 / len(customer_quit)
tmp

,customer_id,ratio
gender,,
F,631,46.740741
M,719,53.259259


In [12]:
tmp = customer_quit[["customer_id", "class_name"]].groupby("class_name").count()
tmp["ratio"] = tmp["customer_id"] * 100 / len(customer_quit)
tmp

,customer_id,ratio
class_name,,
0_종일,664,49.185185
1_주간,344,25.481481
2_야간,342,25.333333


In [13]:
tmp = customer_quit[["customer_id", "campaign_name"]].groupby("campaign_name").count()
tmp["ratio"] = tmp["customer_id"] * 100 / len(customer_quit)
tmp

,customer_id,ratio
campaign_name,,
0_입회비반액할인,368,27.259259
1_입회비무료,273,20.222222
2_일반,709,52.518519


### IV. Analysis All Members

#### 1. Are Reguluar Visiting Members Staying More?

- Grouping uselog data with customer, 'year and month' and weekday.
- Does the member visit a center on specific day regulary stay more longer?

In [14]:
use_log

,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01
...,...,...,...
197423,L00000049209753,TS977703,2019-03-31
197424,L00000049209754,TS979550,2019-03-31
197425,L00000049209755,TS995299,2019-03-31
197426,L00000049209756,TS995853,2019-03-31


- Need to grouping with customer_id and weekday

In [15]:
use_log["weekday"] = pd.to_datetime(use_log["usedate"]).dt.weekday
use_log["ym"] = pd.to_datetime(use_log["usedate"]).dt.strftime("%Y%m")
use_log

,log_id,customer_id,usedate,weekday,ym
0,L00000049012330,AS009373,2018-04-01,6,201804
1,L00000049012331,AS015315,2018-04-01,6,201804
2,L00000049012332,AS040841,2018-04-01,6,201804
3,L00000049012333,AS046594,2018-04-01,6,201804
4,L00000049012334,AS073285,2018-04-01,6,201804
...,...,...,...,...,...
197423,L00000049209753,TS977703,2019-03-31,6,201903
197424,L00000049209754,TS979550,2019-03-31,6,201903
197425,L00000049209755,TS995299,2019-03-31,6,201903
197426,L00000049209756,TS995853,2019-03-31,6,201903


In [16]:
use_log_new = pd.DataFrame(use_log.groupby(["customer_id", "ym", "weekday"]).count()["log_id"])
use_log_new.rename(columns={"log_id": "count"}, inplace=True)
use_log_new

count
customer_id ym     weekday       
AS002855    201804 5            4
            201805 2            1
                   5            4
            201806 5            5
            201807 1            1
...                           ...
TS999855    201901 1            1
                   5            4
                   6            1
            201902 5            4
            201903 5            4

[93333 rows x 1 columns]

- There is a preprocessed data describing that the member' visit times on specific weekday in 'year-month'
- If the member visit the center on specific weekday every week, add regular_flag 1, or 0

In [17]:
use_log_new = use_log_new.groupby(["customer_id", "ym"]).max()
use_log_new

count
customer_id ym           
AS002855    201804      4
            201805      4
            201806      5
            201807      4
            201808      1
...                   ...
TS999855    201811      4
            201812      5
            201901      4
            201902      4
            201903      4

[36842 rows x 1 columns]

In [18]:
use_log_new["reg_flag"] = 1
tmp = use_log_new["reg_flag"].where(use_log_new["count"] >=4, 0)
tmp

customer_id  ym    
AS002855     201804    1
             201805    1
             201806    1
             201807    1
             201808    0
                      ..
TS999855     201811    1
             201812    1
             201901    1
             201902    1
             201903    1
Name: reg_flag, Length: 36842, dtype: int64

In [19]:
tmp = tmp.groupby("customer_id").any()
tmp

customer_id
AS002855     True
AS008805     True
AS009013    False
AS009373     True
AS015233     True
            ...  
TS995853     True
TS998593     True
TS999079     True
TS999231     True
TS999855     True
Name: reg_flag, Length: 4192, dtype: bool

In [20]:
customer_final = pd.merge(customer, tmp, on="customer_id", how="left")
customer_final

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,reg_flag
0,OA832399,XXXX,C01,F,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반,True
1,PL270116,XXXXX,C01,M,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반,True
2,OA974876,XXXXX,C01,M,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반,True
3,HD024127,XXXXX,C01,F,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반,True
4,HD661448,XXXXX,C03,F,2015-05-01,NaT,CA1,0,2_야간,6000,2_일반,True
...,...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,M,2019-03-14,NaT,CA1,0,0_종일,10500,2_일반,False
4188,HD246549,XXXXX,C01,F,2019-03-14,NaT,CA1,0,0_종일,10500,2_일반,False
4189,GD037007,XXXXX,C03,M,2019-03-14,NaT,CA1,0,2_야간,6000,2_일반,False
4190,OA953150,XXXXX,C01,M,2019-03-14,NaT,CA1,0,0_종일,10500,2_일반,False


In [21]:
customer_final.groupby("reg_flag").count()["customer_id"]

reg_flag
False     779
True     3413
Name: customer_id, dtype: int64

- Result: The members who visit the center regularly tend to stay their membership than the members who does not visit regularly.

#### Then, How about the membership period?

In [22]:
customer_final["end_date"].unique()

<DatetimeArray>
[                'NaT', '2018-04-30 00:00:00', '2018-05-31 00:00:00',
 '2018-06-30 00:00:00', '2018-07-31 00:00:00', '2018-08-31 00:00:00',
 '2018-09-30 00:00:00', '2018-11-30 00:00:00', '2018-12-31 00:00:00',
 '2019-01-31 00:00:00', '2018-10-31 00:00:00', '2019-02-28 00:00:00',
 '2019-03-31 00:00:00']
Length: 13, dtype: datetime64[ns]

- Change the NaT in end_date column to "2019-04-30 00:00:00"

In [23]:
customer_final["period"] = customer_final["end_date"].fillna(pd.to_datetime("20190430"))

In [24]:
customer_final["period"].unique()

<DatetimeArray>
['2019-04-30 00:00:00', '2018-04-30 00:00:00', '2018-05-31 00:00:00',
 '2018-06-30 00:00:00', '2018-07-31 00:00:00', '2018-08-31 00:00:00',
 '2018-09-30 00:00:00', '2018-11-30 00:00:00', '2018-12-31 00:00:00',
 '2019-01-31 00:00:00', '2018-10-31 00:00:00', '2019-02-28 00:00:00',
 '2019-03-31 00:00:00']
Length: 13, dtype: datetime64[ns]

In [25]:
for idx in customer_final["period"].index:
    delta = relativedelta(customer_final["period"].iloc[idx], customer_final["start_date"].iloc[idx])
    customer_final["period"].iloc[idx] = delta.years * 12 + delta.months

customer_final

/tmp/ipykernel_14107/2442980824.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  customer_final["period"].iloc[idx] = delta.years * 12 + delta.months
/tmp/ipykernel_14107/2442980824.py:3: SettingWithCopyWarning: 
A value is trying to be s

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,reg_flag,period
0,OA832399,XXXX,C01,F,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반,True,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반,True,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반,True,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaT,CA1,0,0_종일,10500,2_일반,True,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaT,CA1,0,2_야간,6000,2_일반,True,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,M,2019-03-14,NaT,CA1,0,0_종일,10500,2_일반,False,1
4188,HD246549,XXXXX,C01,F,2019-03-14,NaT,CA1,0,0_종일,10500,2_일반,False,1
4189,GD037007,XXXXX,C03,M,2019-03-14,NaT,CA1,0,2_야간,6000,2_일반,False,1
4190,OA953150,XXXXX,C01,M,2019-03-14,NaT,CA1,0,0_종일,10500,2_일반,False,1


In [27]:
customer_final[["reg_flag", "period"]].groupby("reg_flag").agg(["min", "mean", "median", "max"])

period                      
            min       mean median max
reg_flag                             
False         1   6.726573    6.0  23
True          1  21.599766   20.0  47